In [4]:
import json
import os
from typing import Dict, List
from dataclasses import dataclass
from collections import defaultdict

teams = [
    "Complexity", "OG", "HEROIC", "Chinggis Warriors", "B8", "Imperial",
    "BetBoom", "Nemiga", "TYLOO", "NRG", "Lynn Vision", "Legacy",
    "Wildcard", "Metizport", "FlyQuest", "Fluxo", "Falcons", "FAZE",
    "3DMAX", "Virtus.pro", "FURIA", "MIBR", "paiN", "M80",
    "Aurora", "G2", "Natus Vincere", "Liquid", "Vitality", "MOUZ",
    "Spirit", "The MongolZ"
]



In [ ]:
def generate_markdown_report(stage_stats: Dict[str, dict], stage: str) -> str:
    markdown = f"### {stage}\n\n"
    
    if stage_stats:
        markdown += "| Team | W-L | New Wins Against | New Losses Against |\n"
        markdown += "|------|-----|-----------------|------------------|\n"
        
        # Sort teams by wins and losses
        sorted_teams = sorted(
            stage_stats.items(),
            key=lambda x: (x[1]['wins'], -x[1]['losses']),
            reverse=True
        )
        
        for team, stats in sorted_teams:
            markdown += f"| {team} | +{stats['wins']}-{stats['losses']} | "
            markdown += f"{stats['win over'] or '-'} | {stats['loss over'] or '-'} |\n"
    else:
        markdown += "*No new results for this stage*\n"
    
    markdown += "\n"
    return markdown

# Collect all data first
models_data = {}

for model in list_model_directories():
    models_data[model] = {}
    previous_stage_data = {}
    
    for stage in stages:
        stage_stats = {}
        
        for team in teams:
            current_data = get_team_stats(model, team, stage)
            if current_data:
                prev_data = previous_stage_data.get(team)
                
                if prev_data:
                    incremental_data = subtract_previous_results(current_data, prev_data)
                else:
                    incremental_data = current_data
                
                if incremental_data['wins'] > 0 or incremental_data['losses'] > 0:
                    stage_stats[team] = incremental_data
                
                previous_stage_data[team] = current_data
        
        if stage_stats:
            models_data[model][stage] = stage_stats

# Generate and save individual README files for each model
for model, stages_data in models_data.items():
    markdown_content = f"# {model} Results\n\n"
    
    for stage in stages:
        stage_stats = stages_data.get(stage, {})
        markdown_content += generate_markdown_report(stage_stats, stage)
    
    # Save to model-specific README
    readme_path = os.path.join(model, 'README.md')
    with open(readme_path, 'w') as f:
        f.write(markdown_content)
    
    print(f"Created README.md for {model}")

print("\nAll README files have been created")

Current directory: /home/luiz/oss/cs2-match-prediction/new
Found model directories: ['claude-sonnet-4', 'gpt-o4-mini', 'deepseek-chat', 'sabia3', 'claude-opus-4', 'gpt41']
Report has been saved to match_results.md


In [2]:
def list_model_directories():
    current_dir = os.getcwd()
    print(f"Current directory: {current_dir}")
    
    # List all items in the current directory
    model_dirs = [d for d in os.listdir() if os.path.isdir(d) and not d.startswith('stats')]
    print(f"Found model directories: {model_dirs}")
    return model_dirs


In [3]:
for model in list_model_directories():
    print(f"\nProcessing model: {model}")
    for team in teams:
        try:
            file_path = f"{model}/playoffs/championship-cached/{team}.json"
            with open(file_path, 'r') as file:
                data = json.load(file)
                print(f"Successfully read {model}/playoffs/championship-cached/{team}.json")
                print(data)
        except FileNotFoundError:
            print(f"File not found: {model}/{team}.json")
        except json.JSONDecodeError:
            print(f"Invalid JSON format in file: {model}/{team}.json")
        except Exception as e:
            print(f"Error processing {model}/{team}.json: {str(e)}")

Current directory: /home/luiz/oss/cs2-match-prediction/new
Found model directories: ['claude-sonnet-4', 'gpt-o4-mini', 'deepseek-chat', 'sabia3', 'claude-opus-4', 'gpt41']

Processing model: claude-sonnet-4
Successfully read claude-sonnet-4/playoffs/championship-cached/Complexity.json
{'wins': 0, 'losses': 3, 'win over': '', 'loss over': 'OG, Fluxo, NRG'}
Successfully read claude-sonnet-4/playoffs/championship-cached/OG.json
{'wins': 2, 'losses': 3, 'win over': 'Complexity, Chinggis Warriors', 'loss over': 'B8, TYLOO, Imperial'}
Successfully read claude-sonnet-4/playoffs/championship-cached/HEROIC.json
{'wins': 10, 'losses': 2, 'win over': 'Chinggis Warriors, Legacy, TYLOO, Falcons, TYLOO, BetBoom, FAZE, Legacy, The MongolZ, Natus Vincere', 'loss over': 'Vitality, Vitality'}
Successfully read claude-sonnet-4/playoffs/championship-cached/Chinggis Warriors.json
{'wins': 6, 'losses': 4, 'win over': 'NRG, Metizport, FlyQuest, Falcons, paiN, FAZE', 'loss over': 'HEROIC, OG, FURIA, 3DMAX'}
S